In [45]:
import numpy as np
import pickle
import time
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
from datetime import datetime

In [79]:
start_run = 85
end_run = 85

output_file_name = 'Run85_list.pkl'
boards = 3

In [80]:
Tstamp_us = []
Brd = []
Ch = []
LG = []
HG = []
runNum = []

file_start_time = None

for run_number in range(start_run, end_run+1):
    input_file_name = f"Run{run_number}_list.txt"

    with open(input_file_name) as f:
        lines = f.read().split('\n')

    last_tstamp = 0  # Initialize last timestamp for continuity across files

    for i, line in enumerate(lines):
        if i == 6:  # Line containing the start time
            start_time_str = ' '.join(line.split()[4:-1])  # Extract start time string, excluding "UTC"
            file_start_time = datetime.strptime(start_time_str, "%a %b %d %H:%M:%S %Y")
            file_start_time = file_start_time.timestamp() * 1e6  # Convert to microseconds
        elif i > 8:
            data = line.split()

            if len(data) == 6:
                Tstamp_us.append(float(data[0]) + file_start_time)
                last_tstamp = float(data[0]) + file_start_time
                Brd.append(int(data[2]))
                Ch.append(int(data[3]))
                LG.append(int(data[4]))
                HG.append(int(data[5]))
                runNum.append(run_number)
            elif len(data) == 4:
                if last_tstamp is not None:
                    Tstamp_us.append(last_tstamp)
                Brd.append(int(data[0]))
                Ch.append(int(data[1]))
                LG.append(int(data[2]))
                HG.append(int(data[3]))
                runNum.append(run_number)

# Convert timestamps to datetime objects
Tstamp_utc = [datetime.utcfromtimestamp(tstamp_us / 1e6) for tstamp_us in Tstamp_us]


In [81]:
window_size = 100
current_event_id = 1
current_timestamp = Tstamp_us[0]
event_ids = []
for timestamp in Tstamp_us:
    if timestamp - current_timestamp <= window_size:
        event_ids.append(current_event_id)
    else:
        current_event_id += 1
        event_ids.append(current_event_id)
        current_timestamp = timestamp
        
event_counts = Counter(event_ids)
mask = [event_counts[event_id] == (boards*64) for event_id in event_ids]

In [82]:
Brd = [value for i, value in enumerate(Brd) if mask[i]]
Ch = [value for i, value in enumerate(Ch) if mask[i]]
LG = [value for i, value in enumerate(LG) if mask[i]]
HG = [value for i, value in enumerate(HG) if mask[i]]
Tstamp_us = [value for i, value in enumerate(Tstamp_us) if mask[i]]
Tstamp_utc = [value for i, value in enumerate(Tstamp_utc) if mask[i]]
event_ids = [value for i, value in enumerate(event_ids) if mask[i]]

In [83]:
class SiPM:
    def __init__(self, layer, board, sipm):
        self.layer = layer
        self.board = board
        self.sipm = sipm

class SipmPos:
    def __init__(self, x, y):
        self.x = x
        self.y = y

nLayers = 11
nLayerBoards = 4
nHexBoardChannels = 7
nSqaBoardChannels = 4
nHexLayers = 4
nHexBoards = 4 * nHexLayers
nHexChannels = 7 * nHexBoards
nEightLayerChannels = 4 * nLayerBoards * nHexBoardChannels + 4 * nLayerBoards * nSqaBoardChannels
nNineLayerChannels = nEightLayerChannels + 3 * 4
nTenLayerChannels = nEightLayerChannels + 2 * 3 * 4
maxChannel = 192 - 1

mm = 1
cm = 10 * mm

boardLabel = [
    # top right, top left, bottom left, bottom right
    [1, 2, 3, 4],    # hexagon
    [5, 6, 7, 8],    # hexagon
    [10, 9, 11, 12], # hexagon
    [26, 13, 25, 28],# hexagon
    [30, 41, 14, 37],# transition: top 2 square, bottom 2 hexagon
    [38, 54, 21, 18],# square
    [44, 22, 58, 47],# square
    [50, 57, 33, 31],# square
    [0, 15, 46, 35], # square
    [0, 20, 45, 36], # square
    [42, 49, 23, 16],# square
    [17, 52, 19, 51],# square
    [55, 43, 32, 39],# square
]

hexBoardSipmPos = [
    SipmPos(50.01 * mm, 80.90 * mm),
    SipmPos(77.64 * mm, 64.95 * mm),
    SipmPos(22.37 * mm, 64.95 * mm),
    SipmPos(50.01 * mm, 48.99 * mm),
    SipmPos(77.64 * mm, 33.04 * mm),
    SipmPos(22.37 * mm, 33.04 * mm),
    SipmPos(50.01 * mm, 17.08 * mm),
]

sqaBoardSipmPos = [
    SipmPos(73.9 * mm, 72.89 * mm),
    SipmPos(26.1 * mm, 72.89 * mm),
    SipmPos(73.9 * mm, 25.09 * mm),
    SipmPos(26.1 * mm, 25.09 * mm),
]

otherHexBoardSipmPos = hexBoardSipmPos[::-1]  # Reversed list

otherSqaBoardSipmPos = sqaBoardSipmPos[::-1]  # Reversed list

def get_sipm(ch):
    if ch < 0 or ch > maxChannel:
        return False

    rest_ch = 0
    pcb = -1
    sipm = -1
    if ch < nHexChannels:
        pcb = ch // nHexBoardChannels
        sipm = ch % nHexBoardChannels
    elif ch < nEightLayerChannels:
        rest_ch = ch - nHexChannels
        pcb = nHexBoards + rest_ch // nSqaBoardChannels
        sipm = rest_ch % nSqaBoardChannels
    elif ch < nNineLayerChannels:
        rest_ch = ch - nEightLayerChannels
        pcb = 8 * nLayerBoards + 1 + rest_ch // nSqaBoardChannels
        sipm = rest_ch % nSqaBoardChannels
    else:
        rest_ch = ch - nNineLayerChannels
        pcb = 10 * nLayerBoards + rest_ch // nSqaBoardChannels
        sipm = rest_ch % nSqaBoardChannels

    layer = pcb // nLayerBoards
    bd = pcb % nLayerBoards
    return SiPM(layer, bd, sipm)

def get_sipm_pos(ch):
    sp = get_sipm(ch)
    if not sp:
        return False

    bl = boardLabel[sp.layer][sp.board]
    if bl == 25:  # left side hexagonal
        x, y = otherHexBoardSipmPos[sp.sipm].x, otherHexBoardSipmPos[sp.sipm].y
    elif bl in (28, 37):  # right side hexagonal, flip the index
        x, y = otherHexBoardSipmPos[-1 - sp.sipm].x, otherHexBoardSipmPos[-1 - sp.sipm].y
    elif bl in (21, 15, 46):  # left side square
        x, y = otherSqaBoardSipmPos[sp.sipm].x, otherSqaBoardSipmPos[sp.sipm].y
    elif bl in (18, 35, 38):  # right side square, flip the index
        x, y = otherSqaBoardSipmPos[-1 - sp.sipm].x, otherSqaBoardSipmPos[-1 - sp.sipm].y
    elif sp.layer < nHexLayers:  # general hex tile
        x, y = hexBoardSipmPos[sp.sipm].x, hexBoardSipmPos[sp.sipm].y
    else:  # general square tile
        x, y = sqaBoardSipmPos[sp.sipm].x, sqaBoardSipmPos[sp.sipm].y

    quadrant = boardLabel[sp.layer][sp.board] % 4
    if quadrant == 0:
        x = x
        y = 99.27 - y
    elif quadrant == 1:
        x = -1 * x
        y = 1.27 + y
    elif quadrant == 2:
        x = -1 * x
        y = -99.27 + y
    elif quadrant == 3:
        x = x
        y = -1.27 - y

    return x, y, 24.0526 + ((sp.layer-1)*27.1526)

In [84]:
x_pos = np.array([get_sipm_pos(ch + 64 * brd)[0] for ch, brd in zip(Ch, Brd)])
y_pos = np.array([get_sipm_pos(ch + 64 * brd)[1] for ch, brd in zip(Ch, Brd)])
z_pos = np.array([get_sipm_pos(ch + 64 * brd)[2] for ch, brd in zip(Ch, Brd)])

In [85]:
events = pd.DataFrame({
    'Brd': Brd,
    'Ch': Ch,
    'LG': LG,
    'HG': HG,
    'Tstamp_us': Tstamp_us,
    'Tstamp_utc': Tstamp_utc,
    'event_ids': event_ids,
    'x_pos': x_pos,
    'y_pos': y_pos,
    'z_pos': z_pos,
})

In [86]:
with open(output_file_name, 'wb') as handle:
    pickle.dump(events, handle, protocol=pickle.HIGHEST_PROTOCOL)